In [1]:
import pandas as pd
import numpy as np
import glob
import datetime
import os

os.getcwd()
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi




BigLotsAccount_id="act_271491453638620"

jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token="EAAkEB3l7ONYBAHsJtDc00ZBtZAW2ye9CRdKsy9ivLlzvgIMsEkZAyZA3mRpuGBF8IvZAR9obyYZCdzfw455nzMEW76A5ZBZBS6Fft2nQsUSs8sZCcGDZBZCbVeiCq0LMvekWjnnDZCuglkNihTAZAWIWTpvzaZBeZCKYywrPl0ZAsoWul36cyQZDZD"

In [12]:
def creatfbaudience(file_path,ACCESS_TOKEN,accountid,description,audience_name):
    '''
    input:
    file_path: str, full_path, basename as the audience name on FB
        make sure the columns contains the "email_address_hash", use this col only
    
    ACCESS_TOKEN: personal token #app id not needed
    accountid: account_id starting with "act_", followed by the id str
    as Nov. 20, 2019, latest verision is 'v6.0'
    '''
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v6.0')
    myAccount = AdAccount(accountid)
    fields = []
    fbname=audience_name
    params = {
      'name': fbname,
      'subtype': 'CUSTOM',
      'description': description,
      'customer_file_source': 'USER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    
    df=pd.read_csv(file_path,dtype=str,)
    listlen = df.shape[0]
    requestUrl = 'https://graph.facebook.com/v6.0/'+ fbname + '/users'
    
    chunck_size=50000
    i_counter=0
    for df_chunck in pd.read_csv(file_path,chunksize=chunck_size,dtype=str,usecols=["email_address_hash"]):
        payload = {}
        payload['schema'] = ['EMAIL']
        df_chunck=df_chunck.rename(columns={"email_address_hash":"EMAIL"})
        df_chunck=df_chunck.drop_duplicates()
        payload_data=df_chunck['EMAIL'].apply(lambda x: [x]).tolist()

        payload['data'] = payload_data
        params = {}
        params ={'payload':payload} 
        CustomAudience(fbname).create_user(
                      fields=fields,
                      params=params,
        )
        i_counter+=1
        print(audience_name,i_counter,datetime.datetime.now())
  

    return 'All done: '+str(datetime.datetime.now())

In [3]:
df_mapping=pd.read_excel("./Blue_cells_only.xlsx",
                         sheet_name="Sheet1",dtype=str,usecols=['segment','Placement Name'])
print(df_mapping['Placement Name'].nunique(),df_mapping['segment'].nunique(),df_mapping.shape)
dict_rename_seg=df_mapping.set_index("segment").to_dict()['Placement Name']
df_mapping.head(2)

10 136 (136, 2)


,segment,Placement Name
0,Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_...,Legacy_NA_Rewards_ActiveM&L_Above50k_Multi_NA
1,Legacy_Active_L_P_Above50K_MultiFamilyOrCondo_...,Legacy_NA_Rewards_ActiveM&L_Above50k_Multi_NA


In [4]:
dict_all_new_df={}
for old_seg,new_seg in dict_rename_seg.items():
    df=pd.read_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q1/upload_files/T_"+old_seg+".csv",
                  dtype=str,usecols=['customer_id_hashed','email_address_hash'])
    if new_seg in dict_all_new_df.keys():
        dict_all_new_df.update({new_seg:dict_all_new_df[new_seg].append(df)})
    else:
        dict_all_new_df.update({new_seg:df})

    # print(old_seg,df.shape)

In [5]:
new_seg_unique_email_count={}
for new_seg,df in dict_all_new_df.items():
    print(new_seg,df.shape,df['customer_id_hashed'].nunique(),df['email_address_hash'].nunique())
    df.to_csv("./"+new_seg+".csv",index=False)
    new_seg_unique_email_count.update({new_seg:df['email_address_hash'].nunique()})

Legacy_NA_Rewards_ActiveM&L_Above50k_Multi_NA (184406, 2) 184406 184406
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA (2815780, 2) 2815780 2815780
Legacy_NA_Rewards_ActiveM&L_Below50k_Multi_NA (197333, 2) 197333 197333
Legacy_NA_Rewards_ActiveM&L_Below50k_Single_NA (803814, 2) 803814 803814
Legacy_NA_Rewards_ActiveM&L_Unknown_Unknown_NA (1268693, 2) 1268693 1268693
SOTF_NA_ActiveM&L_Active_Above50k_Multi_NA (62178, 2) 62178 62178
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA (1285356, 2) 1285356 1285356
SOTF_NA_Rewards_ActiveM&L_Below50k_Multi_NA (80757, 2) 80757 80757
SOTF_NA_Rewards_ActiveM&L_Below50k_Single_NA (331490, 2) 331490 331490
SOTF_NA_Rewards_ActiveM&L_Unknown_Unknown_NA (627265, 2) 627265 627265


In [6]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_aggregated_audience_to_FB_20200317'

In [7]:
list_new_seg_to_up_load=glob.glob("/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_aggregated_audience_to_FB_20200317/*.csv")
print(len(list_new_seg_to_up_load))

10


In [8]:
list_new_seg_to_up_load

['/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_aggregated_audience_to_FB_20200317/Legacy_NA_Rewards_ActiveM&L_Above50k_Multi_NA.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_aggregated_audience_to_FB_20200317/Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_aggregated_audience_to_FB_20200317/Legacy_NA_Rewards_ActiveM&L_Below50k_Multi_NA.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_aggregated_audience_to_FB_20200317/Legacy_NA_Rewards_ActiveM&L_Below50k_Single_NA.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_aggregated_audience_to_FB_20200317/Legacy_NA_Rewards_ActiveM&L_Unknown_Unknown_NA.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/JT_aggregated_audience_to_FB_20200317/SOTF_NA_ActiveM&L_Active_Above50k_Multi_NA.csv',
 '/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_request

In [13]:
for local_file in list_new_seg_to_up_load:
    new_seg_name=os.path.basename(local_file).split(".")[0]
    str_unique_email=str(new_seg_unique_email_count[new_seg_name])
    creatfbaudience(file_path=local_file,
                    ACCESS_TOKEN=jian_token,
                    accountid=BigLotsAccount_id,
                    description="aggregated_emails_"+new_seg_name+"_"+str_unique_email,
                    audience_name=os.path.basename(local_file).split(".")[0]
                   )

Legacy_NA_Rewards_ActiveM&L_Above50k_Multi_NA 1 2020-03-17 19:12:02.657085
Legacy_NA_Rewards_ActiveM&L_Above50k_Multi_NA 2 2020-03-17 19:12:09.628284
Legacy_NA_Rewards_ActiveM&L_Above50k_Multi_NA 3 2020-03-17 19:12:12.153937
Legacy_NA_Rewards_ActiveM&L_Above50k_Multi_NA 4 2020-03-17 19:12:13.897856
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 1 2020-03-17 19:12:23.790503
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 2 2020-03-17 19:12:30.920728
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 3 2020-03-17 19:12:37.470029
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 4 2020-03-17 19:12:39.793537
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 5 2020-03-17 19:12:43.376970
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 6 2020-03-17 19:12:45.388845
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 7 2020-03-17 19:12:51.985508
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 8 2020-03-17 19:12:54.381088
Legacy_NA_Rewards_ActiveM&L_Above50k_Single_NA 9 2020-03-17 19:13:00.989474
Legacy_NA_Reward

Legacy_NA_Rewards_ActiveM&L_Unknown_Unknown_NA 26 2020-03-17 19:20:00.991831
SOTF_NA_ActiveM&L_Active_Above50k_Multi_NA 1 2020-03-17 19:20:03.998779
SOTF_NA_ActiveM&L_Active_Above50k_Multi_NA 2 2020-03-17 19:20:06.151658
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 1 2020-03-17 19:20:13.335798
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 2 2020-03-17 19:20:15.247715
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 3 2020-03-17 19:20:22.006540
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 4 2020-03-17 19:20:28.756284
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 5 2020-03-17 19:20:31.605796
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 6 2020-03-17 19:20:33.575478
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 7 2020-03-17 19:20:35.960076
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 8 2020-03-17 19:20:43.033871
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 9 2020-03-17 19:20:44.912990
SOTF_NA_Rewards_ActiveM&L_Above50k_Single_NA 10 2020-03-17 19:20:46.941219
SOTF_NA_Rewards_ActiveM&L_Above50k_Sin